# **Utilisation de *StackAPI* pour récupérer des questions sur *Stack Overflow***

In [1]:
from stackapi import StackAPI  # API pour récupérer les questions
import pandas as pd  # Pour utiliser un dataframe
from datetime import datetime  # Utilisé lors du filtrage par date

In [2]:
SITENAME = 'stackoverflow'  # Site sur lequel seront faites les requêtes
SITE = StackAPI(SITENAME)  # Instanciation de l'API

Nous allons utiliser *'search/advanced'* qui permet d'avoir plus de paramètres pour filtrer que *'questions'*. Nous nous basons sur les questions récentes (depuis le 1er janvier 2023).

[Documentation de la méthode *'search/advanced'*](https://api.stackexchange.com/docs/advanced-search)

In [3]:
SITE.page_size = 50 # Nombre souhaité de résultats. Au maximum 100, si besoin de plus, il faut incrémenter 'max_pages'
SITE.max_pages = 1 # Nombre de pages

questions = SITE.fetch('search/advanced',  # Permet d'avoir plus de paramètres pour filtrer que 'questions'
                      filter='withbody',  # Pour que le corps du message soit également récupéré
                      sort='votes',  # Classer les questions retournées par nombre de votes
                      order='desc',  # Du plus grand au plus petit
                      min=10,  # Valeur minimum requise de l'élément sur lequel le classement est basé (ici les votes)
                      views=100,  # Nombre de vues minimum
                      answers=1,  # Nombre de réponses minimum
                      fromdate=datetime(2023,1,1),  # Date de création minimum
                      todate=datetime.now()  # Date de création maximum (ici la date actuelle)
)

elements_a_garder = ['tags', 'title', 'body']  # Éléments qui nous intéressent

data = pd.DataFrame(questions['items'])[elements_a_garder]  # Création d'un dataframe avec les données récupérées
display(data)

print('Quota journalier de requêtes restant :', questions['quota_remaining'])  # Affichage du nombre restant de requêtes possibles aujourd'hui (basé sur l'adresse IP)

,tags,title,body
0,"[git, github, ssh, ssh-keys]",Why are connections to GitHub over SSH throwin...,<p>Just sometime ago I started getting this wa...
1,"[ios, xcode, xcode14]",Missing file libarclite_iphoneos.a (Xcode 14.3),<p>After installing Xcode 14.3 in order to run...
2,"[java, android, kotlin, dependencies]",Duplicate class in Kotlin Android,"<p><img src=""https://i.stack.imgur.com/ugp7v.p..."
3,"[android, android-studio]",No matching variant of com.android.tools.build...,<p>I've just created a brand new material 3 co...
4,"[python, pandas, dataframe, attributeerror]",Error &quot;&#39;DataFrame&#39; object has no ...,<p>I am trying to append a dictionary to a Dat...
5,"[build, runtime, web-frontend]",What is causing the warning &#39;Removing intr...,<p>I am getting following <code>warning</code>...
6,"[eslint, prettier]",Prettier 3.0.0 TypeError: prettier.resolveConf...,<p>The error triggered after I run this comman...
7,"[python, error-handling, pip]",How do I solve &quot;error: externally-managed...,<p>Error message:</p>\n<blockquote>\n<pre clas...
8,"[c, function, macros, glib, function-declaration]",What&#39;s the significance of a C function de...,"<p>In <a href=""https://gitlab.gnome.org/GNOME/..."
9,"[ios, xcode, react-native, xcode14]",Use of bitwise &#39;|&#39; with boolean operan...,<p>Just updated my XCode to Version 14.3 today...


Quota journalier de requêtes restant : 62
